In [1]:
import sqlite3
import pandas as pd

In [2]:
db_path = 'F:\pydata\dataset\CBDB_aw_20180831_sqlite.db'

In [8]:
sql = '''
SELECT a.c_assoc_code
    , b.c_assoc_desc_chn
    , COUNT(a.c_assoc_code) count
FROM assoc_data a
LEFT JOIN assoc_codes b 
    ON a.c_assoc_code = b.c_assoc_code
GROUP BY a.c_assoc_code
ORDER BY count DESC
'''

assoc = pd.read_sql(sql, con=sqlite3.connect(db_path))

In [9]:
assoc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 3 columns):
c_assoc_code        452 non-null int64
c_assoc_desc_chn    450 non-null object
count               452 non-null int64
dtypes: int64(2), object(1)
memory usage: 10.7+ KB


In [10]:
assoc.head()

,c_assoc_code,c_assoc_desc_chn,count
0,43,墓誌銘由Y所作,10258
1,44,為Y作墓誌銘,10206
2,32,為Y所著書作序,4977
3,33,書序由Y所作,4977
4,45,書跋由Y所作,4069


In [28]:
sql = '''
SELECT a.c_personid
    , b1.c_name_chn
    , c_assoc_id
    , b2.c_name_chn
    , a.c_assoc_code
    , c.c_assoc_desc_chn
FROM assoc_data a
LEFT JOIN biog_main b1 
    ON a.c_personid = b1.c_personid
LEFT JOIN biog_main b2 
    ON a.c_assoc_id = b2.c_personid
LEFT JOIN assoc_codes c 
    ON a.c_assoc_code = c.c_assoc_code
WHERE (a.c_personid = '3257'
    OR c_assoc_id = '3257')
    AND a.c_assoc_code IN ('9', '564', '548', '231', '232', '117', '27')
'''

person_assoc = pd.read_sql(sql, con=sqlite3.connect(db_path))

In [29]:
person_assoc.head(100)

,c_personid,c_name_chn,c_assoc_id,c_name_chn,c_assoc_code,c_assoc_desc_chn
0,3257,朱熹,4103,顏度,9,友
1,3257,朱熹,7164,張栻,9,友
2,3257,朱熹,10049,蔡元定,9,友
3,3257,朱熹,10083,崔嘉彥,9,友
4,3257,朱熹,10613,林成季,9,友
5,3257,朱熹,11833,劉玶,9,友
6,3257,朱熹,13651,陳宋霖,9,友
7,3257,朱熹,13773,張杰,9,友
8,3257,朱熹,16692,王炎,9,友
9,3257,朱熹,17527,吳儆,9,友
